In [15]:
import pandas as pd

In [ ]:
from ml_investing_wne.data_engineering.crypto_factory import CryptoFactory

In [3]:
asset = CryptoFactory('Bitstamp', 'ETHUSD')

In [4]:
FIRST_COLUMNS = ['datetime', 'price', 'volume']

In [4]:
asset.df

,datetime,price,volume,s,d,side,buy_id,sell_id
9035029,2019-04-03 13:08:15.913,170.38,3.300000,ethusd,2019-04-03,0,3068696456,3068689528
9035030,2019-04-03 13:08:22.715,170.38,5.652455,ethusd,2019-04-03,0,3068696910,3068689528
9035031,2019-04-03 13:08:23.048,170.38,6.214196,ethusd,2019-04-03,0,3068696932,3068689528
9035032,2019-04-03 13:08:23.556,170.38,6.078305,ethusd,2019-04-03,0,3068696970,3068689528
9035033,2019-04-03 13:08:42.824,170.38,1.626045,ethusd,2019-04-03,0,3068698401,3068689528
...,...,...,...,...,...,...,...,...
1054642,2021-11-26 11:59:47.412,3984.78,0.300000,ethusd,2021-11-26,0,1429884980244483,1429884980162568
1054643,2021-11-26 11:59:47.412,3984.79,2.200000,ethusd,2021-11-26,0,1429884980244483,1429884979412997
1054644,2021-11-26 11:59:51.044,3982.87,0.627180,ethusd,2021-11-26,1,1429884994846721,1429884995121153
1054645,2021-11-26 11:59:54.297,3982.52,0.450000,ethusd,2021-11-26,1,1429885008068609,1429885008445441


In [7]:
btc_binance = pd.read_csv('/Users/i0495036/Downloads/BTCUSDT-trades-2017-09.csv', names=['trade_id', 'price', 'volume','quoteQty', 'time', 'is_buyer_maker','is_best_match'])
btc_binance['datetime'] = pd.to_datetime(btc_binance['time'],unit='ms')
rest_of_columns = [x for x in btc_binance.columns if (x not in FIRST_COLUMNS)]
column_order = FIRST_COLUMNS + rest_of_columns
btc_binance = btc_binance[column_order]

In [8]:
btc_binance

,datetime,price,volume,trade_id,quoteQty,time,is_buyer_maker,is_best_match
0,2017-09-01 00:01:00.493,4689.89,0.053417,69180,250.519854,1504224060493,True,True
1,2017-09-01 00:01:00.530,4689.89,0.053417,69181,250.519854,1504224060530,True,True
2,2017-09-01 00:01:00.530,4689.89,0.011315,69182,53.066105,1504224060530,True,True
3,2017-09-01 00:01:00.573,4689.90,0.147065,69183,689.720143,1504224060573,True,True
4,2017-09-01 00:01:18.120,4689.90,0.042047,69184,197.196225,1504224078120,True,True
...,...,...,...,...,...,...,...,...
228740,2017-09-30 23:58:27.117,4346.01,0.002492,297920,10.830257,1506815907117,True,True
228741,2017-09-30 23:58:48.182,4346.01,0.025255,297921,109.758483,1506815928182,True,True
228742,2017-09-30 23:58:48.328,4346.01,0.009227,297922,40.100634,1506815928328,True,True
228743,2017-09-30 23:58:54.388,4378.51,0.003331,297923,14.584817,1506815934388,False,True


In [5]:
btc_binance_big = pd.read_csv('/Users/i0495036/Downloads/BTCUSDT-trades-2023-01.csv', names=['trade_id', 'price', 'volume','quoteQty', 'time', 'is_buyer_maker','is_best_match'])

KeyboardInterrupt: 

In [6]:
btc_binance_big['datetime'] = pd.to_datetime(btc_binance_big['time'],unit='ms')

In [40]:
btc_binance_big['volume'].sum()

7977028.87801001

In [7]:

rest_of_columns = [x for x in btc_binance_big.columns if (x not in CryptoFactory.FIRST_COLUMNS)]
column_order = FIRST_COLUMNS + rest_of_columns
btc_binance_big = btc_binance_big[column_order]

In [12]:
def clean_binance(df):
    df['datetime'] = pd.to_datetime(df['time'],unit='ms')
    df = df[FIRST_COLUMNS]
    return df

In [9]:
import numpy as np

In [10]:
def generate_volumebars(df, frequency=5000):
    
    df_np = df.to_numpy()
    times = df_np[:,0]
    prices = df_np[:,1]
    volumes = df_np[:,2]
    ans =  np.empty([len(prices), 6], dtype=object)
    candle_counter = 0
    vol = 0
    lasti = 0
    for i, _ in enumerate(prices):
        #print(i)
        if i % 10000 == 0:
            print(f'processed {i/len(prices) * 100} %')
        vol += volumes[i]
        if vol >= frequency:
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])        # volume
            candle_counter += 1
            lasti = i+1
            vol = 0
    remaining_df = df_np[i:]
    df = pd.DataFrame(ans[:candle_counter], 
                        columns = ['datetime','open','high', 'low','close', 'volume'])
    return df 

In [ ]:
with pd.read_csv('/Users/i0495036/Downloads/BTCUSDT-trades-2017-09.csv', names=['trade_id', 'price', 'volume','quoteQty', 'time', 'is_buyer_maker','is_best_match'], chunksize=128000) as reader:
    for chunk in reader:
        df_chunk = clean_binance(chunk)
        if isinstance(remaining_df, pd.DataFrame):
            


In [1]:
from ml_investing_wne.binance_data_processor import BinanceDataProcessor

In [2]:
binance_processor = BinanceDataProcessor('/Users/i0495036/Downloads/BTCUSDT-trades-2023-01.csv')

In [3]:
binance_processor.load_chunks()

processed 0.0 %
processed 7.8125 %
processed 15.625 %
processed 23.4375 %
processed 31.25 %
processed 39.0625 %
processed 46.875 %
processed 54.6875 %
processed 62.5 %
processed 70.3125 %
processed 78.125 %
processed 85.9375 %
processed 93.75 %
processed 0.0 %
processed 7.728154439438317 %
processed 15.456308878876634 %
processed 23.184463318314954 %
processed 30.912617757753267 %
processed 38.64077219719159 %
processed 46.36892663662991 %
processed 54.09708107606822 %
processed 61.825235515506535 %
processed 69.55338995494486 %
processed 77.28154439438318 %
processed 85.00969883382149 %
processed 92.73785327325982 %
processed 0.0 %
processed 7.730603914777823 %
processed 15.461207829555645 %
processed 23.191811744333467 %
processed 30.92241565911129 %
processed 38.65301957388911 %
processed 46.383623488666935 %
processed 54.11422740344476 %
processed 61.84483131822258 %
processed 69.5754352330004 %
processed 77.30603914777822 %
processed 85.03664306255605 %
processed 92.76724697733387

In [4]:
binance_processor.processed_df

,datetime,open,high,low,close,volume
0,2023-01-01 00:00:36.668,16541.77,16543.2,16538.45,16540.1,50.07051
1,2023-01-01 00:01:13.620,16540.09,16544.76,16540.03,16543.69,50.00791
2,2023-01-01 00:01:48.358,16543.69,16544.11,16538.48,16538.74,50.01133
3,2023-01-01 00:02:35.728,16538.73,16541.17,16536.41,16536.41,50.28736
4,2023-01-01 00:03:40.320,16536.41,16537.58,16533.92,16534.22,50.0899
...,...,...,...,...,...,...
158417,2023-01-31 23:58:16.177,23139.0,23144.61,23131.44,23140.35,50.01623
158418,2023-01-31 23:58:49.890,23140.3,23144.12,23132.23,23134.43,52.00458
158419,2023-01-31 23:59:14.200,23134.43,23135.34,23127.27,23130.0,53.40376
158420,2023-01-31 23:59:23.909,23130.0,23130.74,23123.31,23123.31,50.09482


In [5]:
(binance_processor.processed_df['volume'].sum() + binance_processor.remaining_df[:,2].sum())/7977028.87801001

1.0000000000000013

In [21]:
7977028.87801001/(31*24)

10721.81300807797

In [11]:
import os
files_path='/Users/i0495036/Documents/sandbox/ml_investing_wne/src/ml_investing_wne/data/raw/crypto/binance_BTCUSDT'

In [12]:
files_path  = os.listdir(files_path)
files_path


['BTCUSDT-trades-2022-12.csv', 'BTCUSDT-trades-2023-01.csv']

In [13]:
type(files_path)

list

In [14]:
files_path.sort()
files_path

['BTCUSDT-trades-2022-12.csv', 'BTCUSDT-trades-2023-01.csv']

In [11]:
df = generate_volumebars(btc_binance_big)

processed 0.0 %
processed 4.371680255306127 %
processed 8.743360510612254 %
processed 13.115040765918382 %
processed 17.486721021224508 %
processed 21.858401276530635 %
processed 26.230081531836763 %
processed 30.60176178714289 %
processed 34.973442042449015 %
processed 39.34512229775514 %
processed 43.71680255306127 %
processed 48.088482808367395 %
processed 52.46016306367353 %
processed 56.831843318979644 %
processed 61.20352357428578 %
processed 65.5752038295919 %
processed 69.94688408489803 %
processed 74.31856434020416 %
processed 78.69024459551028 %
processed 83.06192485081641 %
processed 87.43360510612254 %
processed 91.80528536142867 %
processed 96.17696561673479 %


In [44]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [45]:
x[1:3]

array([1, 2])

In [22]:
df = pd.read_csv('/Users/i0495036/Documents/sandbox/ml_investing_wne/src/ml_investing_wne/data/processed/binance_BTCUSDT/volume_bars_5000.csv', parse_dates=['datetime'])

In [23]:
df

,datetime,open,high,low,close,volume,seconds
0,2018-01-01 13:58:15.807,13715.65,13818.55,12815.70,12994.87,5000.053381,50288.789
1,2018-01-02 02:27:05.502,12994.91,13850.00,12750.00,13503.95,5000.770708,44929.380
2,2018-01-02 09:25:57.879,13514.97,13583.00,12890.02,13432.02,5000.052810,25132.167
3,2018-01-02 16:04:53.133,13423.34,13894.86,13366.80,13769.99,5000.241391,23932.822
4,2018-01-02 19:59:42.272,13769.99,15473.49,13503.00,14710.00,5000.837506,14089.139
...,...,...,...,...,...,...,...
26659,2022-12-31 18:31:37.214,16582.13,16595.66,16571.01,16577.26,5000.004320,4638.902
26660,2022-12-31 19:48:37.060,16577.30,16588.09,16566.37,16583.94,5000.079660,4619.789
26661,2022-12-31 21:26:10.592,16583.95,16590.06,16558.77,16559.76,5000.001280,5853.532
26662,2022-12-31 22:26:20.004,16559.76,16568.34,16506.36,16550.14,5000.052370,3609.412


In [20]:
df.dtypes

datetime    datetime64[ns]
open               float64
high               float64
low                float64
close              float64
volume             float64
seconds            float64
dtype: object

In [41]:
df = pd.read_csv('/Users/i0495036/Documents/sandbox/ml_investing_wne/src/ml_investing_wne/data/raw/crypto/binance_BTCUSDT/BTCUSDT-trades-2018-01.csv', 
                 names=['trade_id', 'price', 'volume','quoteQty', 'time', 'is_buyer_maker','is_best_match'],
                 nrows=100000)

In [42]:
df['datetime'] = pd.to_datetime(df['time'],unit='ms')

In [36]:
df['datetime'].dt.round("60min").max()


Timestamp('2018-01-01 23:00:00')

In [45]:
df['datetime_rounded'] = df['datetime'].dt.floor("30min")

In [46]:
df

,trade_id,price,volume,quoteQty,time,is_buyer_maker,is_best_match,datetime,datetime_rounded
0,4261621,13715.65,0.045474,623.705468,1514764807018,False,True,2018-01-01 00:00:07.018,2018-01-01 00:00:00
1,4261622,13714.95,0.000993,13.618945,1514764809272,False,True,2018-01-01 00:00:09.272,2018-01-01 00:00:00
2,4261623,13714.94,0.092090,1263.008825,1514764813846,False,True,2018-01-01 00:00:13.846,2018-01-01 00:00:00
3,4261624,13714.94,0.010910,149.629995,1514764814918,False,True,2018-01-01 00:00:14.918,2018-01-01 00:00:00
4,4261625,13714.94,0.007091,97.252640,1514764814918,False,True,2018-01-01 00:00:14.918,2018-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...
99995,4361616,13542.00,0.014768,199.988256,1514846082535,True,True,2018-01-01 22:34:42.535,2018-01-01 22:30:00
99996,4361617,13542.01,0.015707,212.704351,1514846082935,True,True,2018-01-01 22:34:42.935,2018-01-01 22:30:00
99997,4361618,13542.01,0.776808,10519.541704,1514846083627,True,True,2018-01-01 22:34:43.627,2018-01-01 22:30:00
99998,4361619,13550.00,0.076440,1035.762000,1514846087176,True,True,2018-01-01 22:34:47.176,2018-01-01 22:30:00
